In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from utility import *
from botorch.acquisition.analytic import ExpectedImprovement,ProbabilityOfImprovement,UpperConfidenceBound

In [2]:
X_train = pd.read_csv('./input/X_train.csv')
y_train = pd.read_csv('./input/y_train.csv').iloc[:,0]
parameter = pd.read_csv('./input/parameters.csv')
score = -pd.read_csv('./input/scores.csv').iloc[:,1:]

In [3]:
log_rmse = lambda yhat,y: np.sqrt(np.mean((np.log(1+y) - np.log(1+yhat))**2))
def xgboost2perf(learning_rate,max_depth,min_child_weight,gamma,alpha,reg_lambda,\
                 subsample,colsample_bytree,objective,tree_method):

    xgboost = XGBRegressor(learning_rate=learning_rate,\
                           n_estimators=600,\
                           nthread=-1,\
                           
                           max_depth=max_depth,\
                           min_child_weight=min_child_weight,\
                           gamma=gamma,\
                           alpha=alpha,\
                           reg_lambda=reg_lambda,\
                           subsample=subsample,\
                           colsample_bytree=colsample_bytree,\
                           objective=objective,\
                           tree_method=tree_method
                           )

    score = 0
    n_fold = 6
    kf = KFold(n_fold,shuffle=True,random_state=7)
    for train_index, test_index in kf.split(X_train):
        xgboost.fit(X_train.iloc[train_index],y_train.iloc[train_index]);
        yhat = np.maximum(xgboost.predict(X_train.iloc[test_index]),0)
        score += log_rmse(yhat,y_train.iloc[test_index])
    score /= n_fold
    return -score # min -> max

In [4]:
# 'reg:squaredlogerror' and 'reg:pseudohubererror' does not work
cat_feat = {'objective':{'reg:squarederror':0,'reg:absoluteerror':1,'reg:gamma':2,'count:poisson':3},\
            'tree_method':{'exact':0,'hist':1}}
bounds = [[0.1,0.6],[2,16],[1.0,100.0],\
          [0.0,100.0],[0.0,100.0],[0.0,100.0],\
          [0.25,1.0],[0.25,1.0]]
IsMax = False
integer = [1,]

In [5]:
T = Transform(cat_feat,integer,bounds)

In [6]:
bo_parameters = {'acq_fun': ExpectedImprovement,\
                'acq_kwargs': {},\
                 #'fit': fit_gpytorch_mll,\
                'BaseKernel': MaternKernel,\
                'eps': 0.6,\
                'q': 1,\
                'num_restarts': 12,\
                'raw_samples': 256,\
                'Bo_iter': 100,\
                'verbose': 10
                }

In [7]:
#%debug
# import warnings
# warnings.filterwarnings('ignore')
x_best,y_best,x,y,model = BO(xgboost2perf,parameter,score,T,**bo_parameters)

best val is -0.1569850825272283 at iter 10
best val is -0.15519789704000098 at iter 20
best val is -0.15002993745557375 at iter 30
best val is -0.14243553952720947 at iter 40
best val is -0.14182616571363452 at iter 50
best val is -0.14036883522436364 at iter 60
best val is -0.12171523575568416 at iter 70
best val is -0.12156883301501548 at iter 80
best val is -0.1211020291162996 at iter 90
best val is -0.12072101592264238 at iter 100


In [9]:
pd.concat([x,y],1).sort_values(by=['scores'],ascending=False)

,learning_rate,max_depth,min_child_weight,gamma,alpha,reg_lambda,subsample,colsample_bytree,objective,tree_method,scores
0,0.100000,3,77.122053,0.000000,52.740566,35.671372,0.558517,0.435898,count:poisson,hist,-0.120721
0,0.100000,3,77.618448,0.000000,53.183083,35.082906,0.558507,0.437715,count:poisson,hist,-0.121102
0,0.100000,3,76.864070,0.000000,52.764747,36.122816,0.556498,0.436855,count:poisson,hist,-0.121217
0,0.100000,3,77.866601,0.000000,53.429737,34.975479,0.558275,0.437907,count:poisson,hist,-0.121260
0,0.100000,3,77.014462,0.000000,52.812075,35.904699,0.558154,0.436729,count:poisson,hist,-0.121350
...,...,...,...,...,...,...,...,...,...,...,...
6,0.326054,3,11.919477,28.552929,76.890287,54.607057,0.502535,0.795976,reg:absoluteerror,hist,-0.284340
9,0.427221,3,80.484861,47.357331,82.625859,91.327121,0.375483,0.492813,reg:absoluteerror,exact,-0.400163
1,0.280107,2,79.122748,94.706736,85.902933,7.053940,0.282078,0.522069,reg:absoluteerror,hist,-0.401253
2,0.403225,3,38.475863,72.333209,41.717891,17.955389,0.535492,0.305701,reg:gamma,exact,-0.402244


In [11]:
x_best,y_best,x,y,model = BO(xgboost2perf,x,y,T,**bo_parameters)

best val is -0.12072101592264238 at iter 10
best val is -0.12072101592264238 at iter 20
best val is -0.12072101592264238 at iter 30
best val is -0.12072101592264238 at iter 40
best val is -0.12072101592264238 at iter 50
best val is -0.12072101592264238 at iter 60
best val is -0.12072101592264238 at iter 70
best val is -0.12072101592264238 at iter 80
best val is -0.12072101592264238 at iter 90
best val is -0.12072101592264238 at iter 100


In [36]:
pd.concat([x,y],1).sort_values(by=['scores'],ascending=False).iloc[0:1,:-1].to_csv('best_para.csv',index=False)

In [18]:
x_tor,y_tor = T.forward(x,y)

In [19]:
model = initialize_model(x_tor,y_tor, MaternKernel)
h = model.register_forward_pre_hook(hook_factory(T))

In [20]:
d0 = 8;d1 = 6;
bounds = torch.tensor([[0.0] * d0 + [-10] * d1, [1.0] * d0 + [10] * d1], device=device, dtype=dtype)    
bounds2 = torch.tensor([[0.0] * (d0+d1), [1.0] * (d0+d1)], device=device, dtype=dtype)    

In [21]:
from botorch.acquisition.analytic import AnalyticAcquisitionFunction
class Mean_std(AnalyticAcquisitionFunction):
    def __init__(self,model,beta) -> None:
        super().__init__(model=model)
        self.beta = beta

    def forward(self, X):
        mean, std = self._mean_and_sigma(X, compute_sigma=True)
        return mean - self.beta * std

In [27]:
mean_fun = Mean_std(model,0)
#mean_fun = ExpectedImprovement(model,-y.min().values[0])

In [28]:
#%debug
x_best,y_val = optimize_acqf(mean_fun,bounds2,q=1,num_restarts=256,raw_samples=1024)
print(y_val)
xgboost2perf(*T.backward(x_best[0]))

tensor(0.4412, dtype=torch.float64)


-0.1232718809117837

In [29]:
x_best,y_val = optimize_acqf(mean_fun,bounds,q=1,num_restarts=256,raw_samples=1024)
print(y_val)
xgboost2perf(*T.backward(x_best[0]))

tensor(-3.0380, dtype=torch.float64)


-0.15119561623424996

In [ ]:
mean_fun.

In [240]:
y_val

tensor(0.8417, dtype=torch.float64)

In [231]:
mean_fun(x_best),y_val

(tensor(0.8417, dtype=torch.float64, grad_fn=<SubBackward0>),
 tensor(0.8417, dtype=torch.float64))

In [268]:
ra = torch.rand(1,14)
mean_fun(ra)

tensor(-1.2457, dtype=torch.float64, grad_fn=<SubBackward0>)

In [198]:
ra

tensor([[0.3890, 0.2940, 0.0924, 0.2107, 0.2541, 0.4947, 0.9023, 0.0691, 0.3328,
         0.1652, 0.0317, 0.6529, 0.1729, 0.9956]])

In [ ]:
bo_parameters = {'acq_fun': ProbabilityOfImprovement,\
                'acq_kwargs': {},\
                 #'fit': fit_gpytorch_mll,\
                'BaseKernel': MaternKernel,\
                'eps': 0.6,\
                'q': 1,\
                'num_restarts': 12,\
                'raw_samples': 256,\
                'Bo_iter': 100,\
                'verbose': 10
                }

In [10]:
x_best,y_best,x,y,model = BO(xgboost2perf,x,y,T,**bo_parameters)

best val is 0.12337354963815306 at iter 10
best val is 0.12337354963815306 at iter 20
best val is 0.12337354963815306 at iter 30
best val is 0.12337354963815306 at iter 40
best val is 0.12337354963815306 at iter 50
best val is 0.12279163292531943 at iter 60
best val is 0.12279163292531943 at iter 70
best val is 0.12279163292531943 at iter 80
best val is 0.12279163292531943 at iter 90
best val is 0.12279163292531943 at iter 100
best val is 0.12279163292531943 at iter 110
best val is 0.12279163292531943 at iter 120
best val is 0.12279163292531943 at iter 130
best val is 0.12279163292531943 at iter 140
best val is 0.12279163292531943 at iter 150
best val is 0.12279163292531943 at iter 160
best val is 0.12279163292531943 at iter 170
best val is 0.12279163292531943 at iter 180
best val is 0.12279163292531943 at iter 190
best val is 0.12279163292531943 at iter 200


In [11]:
pd.concat([x,y],1).sort_values(by=['scores'])

,learning_rate,max_depth,min_child_weight,gamma,alpha,reg_lambda,subsample,colsample_bytree,objective,tree_method,scores
0,0.217204,3,4.032077,89.312063,68.737098,42.446055,0.793581,0.863480,count:poisson,exact,0.122792
0,0.112490,3,33.838821,55.609864,35.352550,77.994761,0.847249,0.798098,count:poisson,hist,0.123163
0,0.138227,2,82.783443,87.391227,24.874815,28.739450,0.798733,0.643011,count:poisson,exact,0.123374
0,0.171430,2,63.797034,54.209098,49.586350,49.026413,0.637314,0.486718,count:poisson,hist,0.123925
0,0.109649,2,81.560307,30.995282,64.987504,89.510713,0.639748,0.749652,count:poisson,hist,0.125633
...,...,...,...,...,...,...,...,...,...,...,...
0,0.151224,5,27.591647,70.100847,20.921677,43.097579,0.511635,0.564218,reg:gamma,hist,0.406419
0,0.185521,14,9.883601,54.634533,19.348101,17.742480,0.395278,0.760470,reg:gamma,hist,0.406935
0,0.149578,6,12.780756,56.191086,8.357147,3.998911,0.429913,0.455019,reg:gamma,hist,0.406968
0,0.586300,12,55.966766,58.363899,4.782054,99.838437,0.267627,0.987522,reg:gamma,hist,0.407123


In [12]:
x.to_csv('bo_x.csv'); y.to_csv('bo_y.csv')